In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
from __future__ import print_function
import json

from datetime import date
import datetime
#imports

There are 20 teams in the PL, each plays 38 matches for a total of 760. The first game in the 2022-2023 season is number 18202, and it counts up one by one from there, so the total range will be 18,202 to 18,961. Then we will take only the completed games from there.

NEXT: Will add something to get all of the players from the match, and then count the defensive players to determine back 4 or back 5

Code will be -- take all of the elements where the class "player-position" is DC... get a count, if it's two then the team will be in a back four, if there are 3 then it's a back five with wingbacks

In [ ]:
#As an idea for range: scrape total number of games played from somwhere, save it as a variable games_played and make the range
# 18202, 18202 + games_played

In [52]:
table_url = 'https://www.premierleague.com/tables'

In [75]:
res = requests.get(table_url)
soup = BeautifulSoup(res.content, 'lxml')
table_data = soup.find_all('td', {"class": ""})

In [88]:
table_data

[<td>11</td>,
 <td>9</td>,
 <td>1</td>,
 <td>1</td>,
 <td> 
         +14
 
  </td>,
 <td colspan="13">
 <a class="expandableTeam" href="/clubs/1/Arsenal/overview">
 <span class="badge badge-image-container" data-size="50" data-widget="club-badge-image">
 <img class="badge-image badge-image--50" src="https://resources.premierleague.com/premierleague/badges/50/t3.png" srcset="https://resources.premierleague.com/premierleague/badges/50/t3@x2.png 2x"/>
 </span>
 <span class="teamName">Arsenal</span>
 </a>
 <div class="expandableFixtures">
 <div class="resultWidget">
 <div class="label"><strong>Recent Result</strong> - Sunday 23 October 2022</div>
 <a class="matchAbridged pre" href="/match/75037">
 <span class="teamName"><abbr title="Southampton">SOU</abbr></span>
 <span class="badge badge-image-container" data-size="20" data-widget="club-badge-image">
 <img class="badge-image badge-image--20" src="https://resources.premierleague.com/premierleague/badges/20/t20.png" srcset="https://resource

In [85]:
table_data[0]

<td>11</td>

In [83]:
table_data[12]

<td>12</td>

In [86]:
table_data[24]

<td>11</td>

In [90]:
table_data[36]

<td>12</td>

In [2]:
# d = range(18202,18962)
match_range = range(18202,18400)

In [3]:
urls = []
for i in match_range:
    base_url = f'https://understat.com/match/{i}'
    urls.append(base_url)

In [4]:
all_res_data = []
for i in urls:
    res = requests.get(i)
    soup = BeautifulSoup(res.content, 'lxml')
    scripts = soup.find_all('script')
#     positions = soup.find_all("span", {"class": "player-position"})

    all_res_data.append(scripts)
    #Here we can append only the ones that have 'script'

In [5]:
backup_all_res_data = all_res_data
#Adding a backup unless we need it and so we don't need to scrape again

In [6]:
all_res_data = backup_all_res_data

In [7]:
scripts = []
for i in all_res_data:
    if len(i) > 1:
        scripts.append(i)
    else:
        pass
#All of the incomplete games in all_res_data will have a length of zero, so this is how we will filter out the unplayed games
#if necessary

In [8]:
script_list = []
for i in scripts:
    script_list.append(i[1])
#The [1] element is the one with all of the shot data

In [9]:
for i in range(0, len(script_list)):
    script_list[i] = str(script_list[i])
#turning all of our data into strings

In [10]:
data_list = []

# strip unnecessary symbols and get only JSON data 
for i in range(0, len(script_list)):
    ind_start = script_list[i].index("('")+2 
    ind_end = script_list[i].index("')") 
    json_data = script_list[i][ind_start:ind_end] 
    json_data = json_data.encode('utf8').decode('unicode_escape')

#convert string to json format
    data = json.loads(json_data)
    data_list.append(data)

In [11]:
all_data = []
match_id_list = []
x_list = []
y_list = []
home_score_list = []
away_score_list = []
player_list = []
xG_list = []
h_a_list = []
situation_list = []
result_list = []
team_list = []
opposing_team_list = []
minute_list = []
length = len(data_list)
for i in range(length):
    match_id = [] 
    x = []
    y = []
    home_score = []
    away_score = []
    player = []
    xG = []
    h_a = []
    situation = []
    result = []
    team = []
    opposing_team = []
    minute = []
    data_away = data_list[i]['a']
    data_home = data_list[i]['h']

    for index in range(len(data_home)):
        for key in data_home[index]:
            if key == 'X':
                x.append(data_home[index][key])
            if key == 'Y':
                y.append(data_home[index][key])
            if key == 'h_team':
                team.append(data_home[index][key])
            if key == 'a_team':
                opposing_team.append(data_home[index][key])
            if key == 'xG':
                xG.append(data_home[index][key])
            if key == 'situation':
                situation.append(data_home[index][key])
            if key == 'result':
                result.append(data_home[index][key])
            if key == 'h_goals':
                home_score.append(data_home[index][key])
            if key == 'a_goals':
                away_score.append(data_home[index][key])
            if key == 'minute':
                minute.append(data_home[index][key]) 
            if key == 'player':
                player.append(data_home[index][key])
            if key == 'match_id':
                match_id.append(data_home[index][key])
            if key == 'h_a':
                h_a.append(data_home[index][key])
        
        #NEED TO APPEND EVERYTHING TO ALL_DATA

    for index in range(len(data_away)):
        for key in data_away[index]:
            if key == 'X':
                x.append(data_away[index][key])
            if key == 'Y':
                y.append(data_away[index][key])
            if key == 'a_team':
                team.append(data_away[index][key])
            if key == 'h_team':
                opposing_team.append(data_away[index][key])
            if key == 'xG':
                xG.append(data_away[index][key])
            if key == 'situation':
                situation.append(data_away[index][key])
            if key == 'result':
                result.append(data_away[index][key])
            if key == 'h_goals':
                home_score.append(data_away[index][key])
            if key == 'a_goals':
                away_score.append(data_away[index][key])
            if key == 'minute':
                minute.append(data_away[index][key])
            if key == 'player':
                player.append(data_away[index][key])
            if key == 'match_id':
                match_id.append(data_away[index][key])
            if key == 'h_a':
                h_a.append(data_away[index][key]) 
                
    match_id_list.append(match_id)
    x_list.append(x)
    y_list.append(y)
    team_list.append(team)
    opposing_team_list.append(opposing_team)
    xG_list.append(xG)
    situation_list.append(situation)
    result_list.append(result)
    home_score_list.append(home_score)
    away_score_list.append(away_score)
    minute_list.append(minute)
    player_list.append(player)
    h_a_list.append(h_a)

In [12]:
match_id_list = [val for sublist in match_id_list for val in sublist]
x_list = [val for sublist in x_list for val in sublist]
y_list = [val for sublist in y_list for val in sublist]
team_list = [val for sublist in team_list for val in sublist]
opposing_team_list = [val for sublist in opposing_team_list for val in sublist]
xG_list = [val for sublist in xG_list for val in sublist]
situation_list = [val for sublist in situation_list for val in sublist]
result_list = [val for sublist in result_list for val in sublist]
home_score_list = [val for sublist in home_score_list for val in sublist]
away_score_list = [val for sublist in away_score_list for val in sublist]
minute_list = [val for sublist in minute_list for val in sublist]
player_list = [val for sublist in player_list for val in sublist]
h_a_list = [val for sublist in h_a_list for val in sublist]
#Flattening nested lists

In [13]:
col_names = ['match_id','x','y','xG','minute','situation','result','team','opposing_team','h_a','player','home_score','away_score']
df = pd.DataFrame([match_id_list,x_list,y_list,xG_list,minute_list,situation_list,result_list,team_list,opposing_team_list,h_a_list,player_list,home_score_list,away_score_list],index=col_names)
df = df.T
df = df.astype({'minute':'int'})
df = df.sort_values(by=['match_id','minute'])
df = df.reset_index()

In [14]:
df.isna().sum()
#Checking for null values

index            0
match_id         0
x                0
y                0
xG               0
minute           0
situation        0
result           0
team             0
opposing_team    0
h_a              0
player           0
home_score       0
away_score       0
dtype: int64

# xG

In [15]:
# test_home_list = np.where((df['result'] == 'Goal') & (df['h_a'] == 'h') | (df['result'] == 'OwnGoal') & (df['h_a'] == 'a'), 1, 0)

In [16]:
df['xG'] = df['xG'].astype(float)
df['match_id'] = df['match_id'].astype(int)

In [17]:
xg_total = df.groupby('team')['xG'].sum()
#Count total xG for every team

In [18]:
xg_total = pd.DataFrame(data=xg_total)
# xg_total = xg_total.reset_index(level=0)

In [19]:
xg_total['xGA'] = df.groupby('opposing_team')['xG'].sum()

In [20]:
xg_total['xG_difference'] = xg_total['xG'] - xg_total['xGA']

In [21]:
xg_total['matches_played'] = df.groupby('team')['match_id'].nunique()
#Get number of matches played for each team

In [22]:
xg_total = xg_total.reset_index(level=0)

In [23]:
xg_total['avg_xG_diff'] = round(xg_total['xG_difference'] / xg_total['matches_played'], 2)
xg_total['avg_xG_for'] = round(xg_total['xG'] / xg_total['matches_played'], 2)
xg_total['avg_xGA'] = round(xg_total['xGA'] / xg_total['matches_played'], 2)

In [24]:
xg_total.isna().sum()

team              0
xG                0
xGA               0
xG_difference     0
matches_played    0
avg_xG_diff       0
avg_xG_for        0
avg_xGA           0
dtype: int64

# Non Pen xG

In [25]:
non_pen_xg = df.groupby('team')
non_pen_xg = non_pen_xg.apply(lambda x: x[x['situation'] != 'Penalty']['xG'].sum())
#Getting non pen xG and adding to df
#Not working because this groupby result does not have a name so can't go into a df

In [26]:
non_pen_xga = df.groupby('opposing_team')
non_pen_xga = non_pen_xga.apply(lambda x: x[x['situation'] != 'Penalty']['xG'].sum())

In [27]:
non_pen_xga

opposing_team
Arsenal                     8.679898
Aston Villa                11.659181
Bournemouth                16.112490
Brentford                  15.900137
Brighton                   10.027473
Chelsea                    14.964570
Crystal Palace             15.785795
Everton                    17.068954
Fulham                     22.683780
Leeds                      12.440922
Leicester                  14.778440
Liverpool                  13.510209
Manchester City             7.110206
Manchester United          12.950050
Newcastle United           12.856749
Nottingham Forest          16.963168
Southampton                17.342794
Tottenham                  12.112152
West Ham                   10.999698
Wolverhampton Wanderers    12.896053
dtype: float64

In [28]:
xg_total['non_pen_xG'] = non_pen_xg.values
xg_total['non_pen_xGA'] = non_pen_xga.values

#Now need to match for team

In [29]:
xg_total['xG_difference'] = xg_total['xG'] - xg_total['xGA']

In [30]:
xg_total['non_pen_xg_difference'] = xg_total['non_pen_xG'] - xg_total['non_pen_xGA']

In [31]:
xg_total['avg_non_pen_xG_diff'] = round(xg_total['non_pen_xg_difference'] / xg_total['matches_played'], 2)
xg_total['avg_non_pen_xG_for'] = round(xg_total['non_pen_xG'] / xg_total['matches_played'], 2)
xg_total['avg_non_pen_xGA'] = round(xg_total['non_pen_xGA'] / xg_total['matches_played'], 2)

In [32]:
xg_total = xg_total.sort_values(by='avg_xG_diff', ascending=False).reset_index(drop=True)

In [33]:
xg_total

,team,xG,xGA,xG_difference,matches_played,avg_xG_diff,avg_xG_for,avg_xGA,non_pen_xG,non_pen_xGA,non_pen_xg_difference,avg_non_pen_xG_diff,avg_non_pen_xG_for,avg_non_pen_xGA
0,Manchester City,24.302138,7.871375,16.430763,10,1.64,2.43,0.79,23.540970,7.110206,16.430763,1.64,2.35,0.71
1,Arsenal,20.597946,10.202236,10.395710,10,1.04,2.06,1.02,19.836777,8.679898,11.156879,1.12,1.98,0.87
2,Brighton,18.894658,10.788641,8.106017,10,0.81,1.89,1.08,16.611152,10.027473,6.583679,0.66,1.66,1.00
3,Newcastle United,20.997900,13.617918,7.379982,11,0.67,1.91,1.24,20.236731,12.856749,7.379982,0.67,1.84,1.17
4,Liverpool,21.403107,15.793716,5.609391,10,0.56,2.14,1.58,21.403107,13.510209,7.892898,0.79,2.14,1.35
5,Tottenham,18.313123,12.873321,5.439802,11,0.49,1.66,1.17,16.029617,12.112152,3.917464,0.36,1.46,1.10
6,Manchester United,16.325111,12.950050,3.375061,10,0.34,1.63,1.30,15.563942,12.950050,2.613892,0.26,1.56,1.30
7,West Ham,16.021512,12.522035,3.499476,11,0.32,1.46,1.14,13.738005,10.999698,2.738308,0.25,1.25,1.00
8,Brentford,16.892943,15.900137,0.992806,11,0.09,1.54,1.45,14.609437,15.900137,-1.290700,-0.12,1.33,1.45
9,Leeds,12.981228,13.202090,-0.220863,9,-0.02,1.44,1.47,12.220059,12.440922,-0.220863,-0.02,1.36,1.38


In [34]:
# xg_total.to_excel("prem_xG_data.xlsx")

# In Home Games

# In Away Games